In [1]:
class MapBase(object):
    """
    映射基类

    .. note::
       * API接口仿照<<算法>> P228页 设计
       * 约定映射中的key和value均不能为 ``None``
    """

    def put(self, k, v):
        """
        :return: ``None``
        """
        raise NotImplementedError

    def get(self, k):
        """
        :return: k存在时返回其对应的值, 否则返回 ``None``
        """
        raise NotImplementedError

    def delete(self, k):
        return self.put(k, None)

    def delete_min(self):
        """删除最小键"""
        return self.delete(self.min())

    def delete_max(self):
        """删除最大的键"""
        return self.delete(self.max())

    def min(self):
        """最小的键, 映射为空时返回 ``None``"""
        raise NotImplementedError

    def max(self):
        """最大的键, 映射为空时返回 ``None``"""
        raise NotImplementedError

    def keys_between(self, k1, k2):
        """某两个键之间的所有键的集合(包括k1, k2)"""
        raise NotImplementedError

    def keys(self):
        """映射中所有的键(已排序)"""
        return self.keys_between(self.min(), self.max())

    def floor(self, k):
        """小于等于k的最大键, 映射为空时返回 ``None``"""
        raise NotImplementedError

    def ceiling(self, k):
        """大于等于k的最小键, 映射为空时返回 ``None``"""
        raise NotImplementedError

    def rank(self, k):
        """小于k的键的个数"""
        raise NotImplementedError

    def select(self, n):
        """排名为n的键"""
        raise NotImplementedError

    def size(self):
        """映射长度"""
        return self.size_between(self.min(), self.max())

    def size_between(self, k1, k2):
        """某两个k之间的长度"""
        # k1, k2 同时存在/不存在
        if not k1 or not k2:
            return 0
        if k2 < k1:
            return 0
        if self.contains(k2):
            return self.rank(k2) - self.rank(k1) + 1
        else:
            return self.rank(k2) - self.rank(k1)

    def contains(self, k):
        return self.get(k) != None

    def is_empty(self):
        return self.size() == 0

In [2]:
class MappingTest(object):
    """测试"""

    def __init__(self, mapping_kls):
        self._kls = mapping_kls
        self._str = 'testmapping'

    def _fillup(self, mapping):
        for v, k in enumerate(self._str):
            mapping.put(k, v)
        return mapping

    def new_mapping(self):
        return self._fillup(self._kls())

    def test_map_related(self, mapping=None):
        if mapping is None:
            mapping = self.new_mapping()

        print(mapping, end='\n\n')
        print('size: ', mapping.size())
        print('keys: ', mapping.keys())

    def test_single_key(self, mapping=None):
        if mapping is None:
            mapping = self.new_mapping()

        items = [
            'g',    # 中间元素
            'a',    # 左边界
            't',    # 右边界
            'c',    # 中间且不存在
            '0',    # 小于左边界
            'z',    # 大于右边界
        ]

        print(mapping, end='\n\n')
        print('\t'.join(
            [
                'key', 'exist', 'get', 'rank',
                'select', 'floor', 'ceil'
            ]
        ))
        for x in items:
            print(x, end='\t')
            print(mapping.contains(x), end='\t')
            print(mapping.get(x), end='\t')
            print(mapping.rank(x), end='\t')
            print(mapping.select(mapping.rank(x)), end='\t')
            print(mapping.floor(x), end='\t')
            print(mapping.ceiling(x), end='\t')
            print()

    def test_key_range(self, mapping=None):
        if mapping is None:
            mapping = self.new_mapping()

        items = [
            ('g', 'g'),    # 两个存在且相等
            ('g', 'p'),    # 两个都存在
            ('a', 't'),    # 两个都存在, 且是边界
            ('a', 'z'),    # 右边超出边界
            ('0', 't'),    # 左边超出边界
            ('0', 'z'),    # 左右超出边界
        ]

        print(mapping, end='\n\n')
        for k1, k2 in items:
            print(
                (k1, k2),
                mapping.size_between(k1, k2),
                mapping.keys_between(k1, k2),
            )

    def test_delete(self):
        mapping = self._kls()
        print(mapping, end='\n\n')
        print('del a: ', mapping.delete('a'))
        print()

        mapping.put('a', 1)
        print(mapping, end='\n\n')
        print('del a: ', mapping.delete('a'))
        print('del a: ', mapping.delete('a'))
        print()

        print(mapping)